In [99]:
import json
import pandas as pd
import numpy as np


In [100]:
# Change some pandas display options
pd.set_option("display.max_rows", 10000)
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_colwidth", 0)
pd.set_option("display.width", 1000)
pd.set_option("styler.format.precision", 10)


In [101]:
with open("../data/raw/spaces_dummy_data.json", mode="r") as f:
    file = json.load(f)


In [102]:
spaces_details = pd.json_normalize(file, record_path=["available_dates"], meta=["id"], errors="ignore")

# Change columns type
spaces_details["start"] = spaces_details["start"].astype("datetime64[ns]")
spaces_details["end"] = spaces_details["end"].astype("datetime64[ns]")
spaces_details["cancellable"] = spaces_details["cancellable"].astype(int)


# Split date from time
spaces_details["date"] = spaces_details["start"].dt.to_period("d").astype("datetime64[ns]")
spaces_details["date"] = spaces_details["date"].astype("str")


# Get available time span for each date
spaces_details["time_span"] = (spaces_details["end"] - spaces_details["start"]).astype("timedelta64[h]")

# Correct time span to be 24 for whole day
spaces_details["time_span"] = spaces_details["time_span"].apply(lambda x: x + 1 if x == 23 else x)


spaces_details["cancellable_percent"] = spaces_details["cancellable"] * spaces_details["time_span"]
spaces_details["cancellable_percent"] /= spaces_details["time_span"].sum()

# Drop columns
spaces_details.drop(columns="cancellable", inplace=True)

spaces_details.groupby(["id", "date"]).sum()
spaces_details


C:\Users\user\AppData\Local\Temp\ipykernel_9452\614729445.py:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  spaces_details.groupby(["id", "date"]).sum()


,start,end,id,date,time_span,cancellable_percent
0,2022-01-01 08:00:00,2022-01-01 11:00:00,A1,2022-01-01,3.0,0.016129
1,2022-01-01 14:00:00,2022-01-01 18:00:00,A1,2022-01-01,4.0,0.021505
2,2022-01-02 00:00:00,2022-01-02 23:59:59,A1,2022-01-02,24.0,0.000000
3,2022-01-03 10:00:00,2022-01-03 15:00:00,A1,2022-01-03,5.0,0.000000
4,2022-01-04 00:00:00,2022-01-04 23:59:00,A1,2022-01-04,24.0,0.000000
5,2022-01-05 08:00:00,2022-01-05 14:00:00,A1,2022-01-05,6.0,0.032258
6,2022-01-01 00:00:00,2022-01-01 23:59:00,A2,2022-01-01,24.0,0.000000
7,2022-01-02 09:00:00,2022-01-02 14:00:00,A2,2022-01-02,5.0,0.026882
8,2022-01-03 00:00:00,2022-01-03 23:59:00,A2,2022-01-03,24.0,0.129032
9,2022-01-04 11:00:00,2022-01-04 17:00:00,A2,2022-01-04,6.0,0.000000


In [103]:
# Add encoded time columns

result = np.zeros(shape=(spaces_details.shape[0], 24), dtype="int")

result = pd.DataFrame(result, columns=[i for i in range(24)])

spaces_details = pd.concat([spaces_details, result], axis=1)


def encode(row: pd.Series):
    if row["end"].hour == 23:
        end = row["end"].hour
    else:
        end = row["end"].hour - 1
    start = row["start"].hour

    row.loc[start:end] = 1
    return row


spaces_details = spaces_details.apply(encode, axis=1)


In [104]:
spaces_details_group = spaces_details.groupby(["id", "date"]).sum()


C:\Users\user\AppData\Local\Temp\ipykernel_9452\3474892684.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  spaces_details_group = spaces_details.groupby(["id", "date"]).sum()


In [114]:

# Convert multiindex dataframe to json
data = {level: spaces_details_group.xs(level).to_dict("index") for level in spaces_details_group.index.levels[0]}

# data['A1'].keys()
with open("./jsdf.json", mode="w") as f:
    json.dump(data, f, indent=4)

# D = spaces_details_group.groupby(level=0).apply(lambda df: df.xs(df.name).to_dict()).to_dict()


In [125]:
# Get total time span
spaces_details.groupby('id').sum()[['time_span']].plot.bar(x='time_span')

C:\Users\user\AppData\Local\Temp\ipykernel_9452\3175940057.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  spaces_details.groupby('id').sum()[['time_span']].plot.bar(x='time_span')


ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.